<h1>Predicting severity of car collisions<h1>

Loading the necessary libraries

In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

Reading the dataset

In [4]:
Collisions = pd.read_csv('https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv', low_memory = False)

Having a first look of the dataset

In [7]:
Collisions.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


<h3>Data cleaning<h3>

An initial analysis of the data to see the format a number of features:

In [14]:
Collisions.dtypes

SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYCODE.1      int64
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: objec

A correlation matrix helps to identify redundant information e.g. "COLDETKEY" and "INTKEY" are highly correlated, therefore we need just one of these features.

In [8]:
Collisions.corr(method='pearson', min_periods=1)

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,INTKEY,SEVERITYCODE.1,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,SDOT_COLCODE,SDOTCOLNUM,SEGLANEKEY,CROSSWALKKEY
SEVERITYCODE,1.000000,0.010309,0.017737,0.020131,0.022065,0.022079,0.006553,1.000000,0.130949,0.246338,0.214218,-0.054686,0.188905,0.004226,0.104276,0.175093
X,0.010309,1.000000,-0.160262,0.009956,0.010309,0.010300,0.120754,0.010309,0.012887,0.011304,-0.001752,-0.012168,0.010904,-0.001016,-0.001618,0.013586
Y,0.017737,-0.160262,1.000000,-0.023848,-0.027396,-0.027415,-0.114935,0.017737,-0.013850,0.010178,0.026304,0.017058,-0.019694,-0.006958,0.004618,0.009508
OBJECTID,0.020131,0.009956,-0.023848,1.000000,0.946383,0.945837,0.046929,0.020131,-0.062333,0.024604,0.034432,-0.094280,-0.037094,0.969276,0.028076,0.056046
INCKEY,0.022065,0.010309,-0.027396,0.946383,1.000000,0.999996,0.048524,0.022065,-0.061500,0.024918,0.031342,-0.107528,-0.027617,0.990571,0.019701,0.048179
COLDETKEY,0.022079,0.010300,-0.027415,0.945837,0.999996,1.000000,0.048499,0.022079,-0.061403,0.024914,0.031296,-0.107598,-0.027461,0.990571,0.019586,0.048063
INTKEY,0.006553,0.120754,-0.114935,0.046929,0.048524,0.048499,1.000000,0.006553,0.001886,-0.004784,0.000531,-0.012929,0.007114,0.032604,-0.010510,0.018420
SEVERITYCODE.1,1.000000,0.010309,0.017737,0.020131,0.022065,0.022079,0.006553,1.000000,0.130949,0.246338,0.214218,-0.054686,0.188905,0.004226,0.104276,0.175093
PERSONCOUNT,0.130949,0.012887,-0.013850,-0.062333,-0.061500,-0.061403,0.001886,0.130949,1.000000,-0.023464,-0.038809,0.380523,-0.128960,0.011784,-0.021383,-0.032258
PEDCOUNT,0.246338,0.011304,0.010178,0.024604,0.024918,0.024914,-0.004784,0.246338,-0.023464,1.000000,-0.016920,-0.261285,0.260393,0.021461,0.001810,0.565326


From the data description we can choose the features that may be used for a classification. In this study I will mainly focus on numerical features or features that can be converted to a numerical. The other I will dismiss for the purpose of the classification

In [49]:
Collisions_red = Collisions.drop(['OBJECTID', 'INCKEY', 'COLDETKEY', 'ADDRTYPE', 'LOCATION','EXCEPTRSNCODE','EXCEPTRSNDESC', 'SEVERITYCODE.1','SEVERITYDESC','COLLISIONTYPE','PERSONCOUNT','PEDCOUNT','PEDCYLCOUNT','VEHCOUNT','REPORTNO','INCDATE','INCDTTM','INATTENTIONIND','UNDERINFL','PEDROWNOTGRNT','HITPARKEDCAR','SDOT_COLCODE','SDOT_COLDESC','STATUS','SDOTCOLNUM','CROSSWALKKEY','ST_COLCODE','ST_COLDESC','SEGLANEKEY','SPEEDING','INTKEY','JUNCTIONTYPE'], axis=1)

In [52]:
Collisions_red.head(15)

,SEVERITYCODE,X,Y,WEATHER,ROADCOND,LIGHTCOND
0,2,-122.323148,47.703140,Overcast,Wet,Daylight
1,1,-122.347294,47.647172,Raining,Wet,Dark - Street Lights On
2,1,-122.334540,47.607871,Overcast,Dry,Daylight
3,1,-122.334803,47.604803,Clear,Dry,Daylight
4,2,-122.306426,47.545739,Raining,Wet,Daylight
5,1,-122.387598,47.690575,Clear,Dry,Daylight
6,1,-122.338485,47.618534,Raining,Wet,Daylight
7,2,-122.320780,47.614076,Clear,Dry,Daylight
8,1,-122.335930,47.611904,Clear,Dry,Daylight
9,2,-122.384700,47.528475,Clear,Dry,Daylight


In [46]:
Collisions_red.dtypes

SEVERITYCODE      int64
X               float64
Y               float64
JUNCTIONTYPE     object
WEATHER          object
ROADCOND         object
LIGHTCOND        object
dtype: object

Clean: delete the data where there is no information

In [84]:
Collisions_red.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
Collisions_red

,SEVERITYCODE,X,Y,WEATHER,ROADCOND,LIGHTCOND
0,2,-122.323148,47.703140,0,Wet,Daylight
1,1,-122.347294,47.647172,1,Wet,Dark - Street Lights On
2,1,-122.334540,47.607871,0,Dry,Daylight
3,1,-122.334803,47.604803,3,Dry,Daylight
4,2,-122.306426,47.545739,1,Wet,Daylight
...,...,...,...,...,...,...
194668,2,-122.290826,47.565408,3,Dry,Daylight
194669,1,-122.344526,47.690924,1,Wet,Daylight
194670,2,-122.306689,47.683047,3,Dry,Daylight
194671,2,-122.355317,47.678734,3,Dry,Dusk


In [85]:
weather = {'Overcast': 0, 'Raining': 1, 'Clear': 3,'Unknown': 4,'Snowing': 5,'Fog/Smog/Smoke': 6, 'Sleet/Hail/Freezing Rain': 7, 'Blowing Sand/Dirt': 8,
       'Severe Crosswind': 9, 'Partly Cloudy': 10, 'Other': 11}
#Assign these different key-value pair from above dictiionary to your table
Collisions_red.WEATHER = [weather[item] for item in Collisions_red.WEATHER]
#New table
#Collisions_red.head()

KeyError: 0

In [89]:
condition = {'Wet': 0, 'Dry': 1, 'Unknown': 3,'Snow/Slush': 4,'Ice': 5,'Other': 6, 'Sand/Mud/Dirt': 7, 'Standing Water': 8,
       'Oil': 9}
Collisions_red.ROADCOND = [condition[item] for item in Collisions_red.ROADCOND]

In [93]:
light = {'Daylight': 0, 'Dark - Street Lights On': 1, 'Dark - No Street Lights': 2,
       'Unknown': 3, 'Dusk': 4, 'Dawn': 5, 'Dark - Street Lights Off': 6, 'Other': 7,
       'Dark - Unknown Lighting': 8}
Collisions_red.LIGHTCOND = [light[item] for item in Collisions_red.LIGHTCOND]

In [94]:
Collisions_red

,SEVERITYCODE,X,Y,WEATHER,ROADCOND,LIGHTCOND
0,2,-122.323148,47.703140,0,0,0
1,1,-122.347294,47.647172,1,0,1
2,1,-122.334540,47.607871,0,1,0
3,1,-122.334803,47.604803,3,1,0
4,2,-122.306426,47.545739,1,0,0
...,...,...,...,...,...,...
194668,2,-122.290826,47.565408,3,1,0
194669,1,-122.344526,47.690924,1,0,0
194670,2,-122.306689,47.683047,3,1,0
194671,2,-122.355317,47.678734,3,1,4


To use scikit-learn library, we have to convert the Pandas data frame to a Numpy array:

In [95]:
X = Collisions_red[['WEATHER', 'ROADCOND','LIGHTCOND']] .values  #.astype(float)
X[0:5]

array([[0, 0, 0],
       [1, 0, 1],
       [0, 1, 0],
       [3, 1, 0],
       [1, 0, 0]])

What are our labels?

In [97]:
y = Collisions_red['SEVERITYCODE'].values
y[0:5]

array([2, 1, 1, 1, 2])

## Normalize Data 

Data Standardization give data zero mean and unit variance, it is good practice, especially for algorithms such as KNN which is based on distance of cases:

In [99]:
X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
X[0:5]

array([[-1.65780552, -1.0852763 , -0.58058129],
       [-0.95021877, -1.0852763 ,  0.23666739],
       [-1.65780552,  0.05364787, -0.58058129],
       [ 0.46495473,  0.05364787, -0.58058129],
       [-0.95021877, -1.0852763 , -0.58058129]])

### Train Test Split  

In [100]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (147333, 3) (147333,)
Test set: (36834, 3) (36834,)


## Classification

In [101]:
from sklearn.neighbors import KNeighborsClassifier

In [111]:
k = 4
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
neigh

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [112]:
yhat = neigh.predict(X_test)
yhat[0:5]

array([1, 1, 2, 1, 1])

In [113]:
from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat))

Train set Accuracy:  0.6669856719132848
Test set Accuracy:  0.6672910897540316


## Map visualization

In [114]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



In [128]:
Lat = 47.6
Lon = -122.32

In [132]:
Seattle = folium.Map(location=[Lat, Lon], zoom_start=11)
Seattle

In [124]:
Collisions_red.shape

(184167, 6)

In [137]:
# get the first 200 accidents 
limit = 500
Collisions_red_500 = Collisions_red.iloc[0:limit, :]

In [139]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(Collisions_red_500.Y, Collisions_red_500.X):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
Seattle.add_child(incidents)